<a href="https://colab.research.google.com/github/nisargdoshi9/siamese_network_few_shot_learning/blob/main/Siamese_NW_Fruits360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:

#Configuration environment
import os

os.environ['KAGGLE_USERNAME'] = "nisargx9" # username from the json file
os.environ['KAGGLE_KEY'] = "1e6b4d19094e4587b87ea9e3327aecf9" # key from the json file

In [18]:
cd '/content/drive/MyDrive/Projects/Fruit360'

/content/drive/MyDrive/Projects/Fruit360


In [19]:
!kaggle datasets download -d moltean/fruits

 99% 1.28G/1.28G [00:14<00:00, 141MB/s]
100% 1.28G/1.28G [00:14<00:00, 95.4MB/s]


In [ ]:
# !unzip "/content/drive/MyDrive/Projects/Fruit360/fruits.zip" -d "/content/drive/MyDrive/Projects/Fruit360/fruits"

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import cv2

import tensorflow as tf
import keras


In [29]:
dir = "/content/drive/MyDrive/Projects/Fruit360/fruits/fruits-360_dataset/fruits-360/Training"
split = 0.7
num_files_each_class = 10

folder_list = os.listdir(dir)
print(len(folder_list),": number of categories")

131 : number of categories


## Loading dataset into numpy arrays

In [34]:
x = []
y = []
cat_list = []
y_label = 0
for folder_name in folder_list:
  file_list = os.listdir(dir+'/'+folder_name)
  temp = []
  for file_name in file_list[:num_files_each_class]:
    temp.append(len(x))
    im = cv2.imread(dir+'/'+folder_name+'/'+file_name)
    im = cv2.resize(im,(100,100))
    x.append(im)
    y.append(y_label)
  y_label+=1
  cat_list.append(temp)

cat_list = np.asarray(cat_list)
x = np.asarray(x)/255
y = np.asarray(y)
print('X,y, cat_list shape: ',x.shape,y.shape,cat_list.shape)

X,y, cat_list shape:  (1310, 100, 100, 3) (1310,) (131, 10)


# Train test split: Unlike usual training, only training on 70% classes/categories and keep the rest for test

In [51]:
num_classes = int(len(folder_list)*split)
print('Number of folders to train (approx): ',int(num_classes))
num_files = int(x.shape[0]*split)
print('Number of files to train: ',int(num_files))

#Split
x_train = x[:num_files]
y_train = y[:num_files]
cat_train = cat_list[:num_classes]

x_val = x[num_files:]
y_val = y[num_files:]
cat_test = cat_list[num_classes:]

print("Shape of x_train, y_train, cat_train: ",x_train.shape,y_train.shape,cat_train.shape)
print("Shape of x_val, y_val, cat_test: ",x_val.shape,y_val.shape,cat_test.shape)

Number of folders to train (approx):  91
Number of files to train:  916
Shape of x_train, y_train, cat_train:  (916, 100, 100, 3) (916,) (91, 10)
Shape of x_val, y_val, cat_test:  (394, 100, 100, 3) (394,) (40, 10)


## Batch generation function

In [57]:
def get_batch(batch_size=64):
  temp_x = x_train
  temp_cat_list = cat_train
  start = 0
  end = num_classes
  batch_x = []

  batch_y = np.zeros(batch_size)
  # Setting other half to 0
  batch_y[int(batch_size/2):]=1
  np.random.shuffle(batch_y)

  class_list = np.random.randint(start,end,batch_size)
  batch_x.append(np.zeros((batch_size,100,100,3)))
  batch_x.append(np.zeros((batch_size,100,100,3)))

  for i in range(batch_size):
    batch_x[0][i]=temp_x[np.random.choice(temp_cat_list[class_list[i]])]
    # if y_train=0 pick same class, else if y_train=1 pick any other class
    if batch_y[i]==0:
      batch_x[0][i] = temp_x[np.random.choice(temp_cat_list[class_list[i]])]
    else:
      temp_list = np.append(temp_cat_list[:cat_list[i]].flatten(),
                            temp_cat_list[class_list[i]+1:].flatten())
      batch_x[1][i] = temp_x[np.random.choice(temp_list)]
  return (batch_x, batch_y)

## Sequential Model

In [56]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten, MaxPooling2D
from keras.models import Model,Sequential
from keras.losses import binary_crossentropy
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD, Adam

W_init = keras.initializers.RandomNormal(mean=0.0, stddev = 1e-2)
b_init = keras.initializers.RandomNormal(mean=0.0, stddev = 1e-2)

input_shape = (100,100,3)
left_input = Input(input_shape)
right_input = Input(input_shape)

model = keras.model.Sequential([
  keras.layers.Conv2D(64, (10,10), activation='relu', input_shape=input_shape, kernel_initializer=W_init, bias_initializer=b_init, kernel_regularizer=l2(2e-4)),
  keras.layers.MaxPooling2D(2, 2),
  keras.layers.Conv2D(128, (7,7), activation='relu', kernel_initializer=W_init, bias_initializer=b_init, kernel_regularizer=l2(2e-4)),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(128, (4,4), activation='relu', kernel_initializer=W_init, bias_initializer=b_init, kernel_regularizer=l2(2e-4)),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(256, (4,4), activation='relu', kernel_initializer=W_init, bias_initializer=b_init, kernel_regularizer=l2(2e-4)),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dense(4096, activation='sigmoid', kernel_initializer=W_init, bias_initializer=b_init)
])

encoded_l = model(left_input)
encoded_r = model(right_input)

(2, 64, 100, 100, 3)